In [8]:
import pandas as pd

df = pd.read_csv('../data/FSU-GT-08-24-24-PLAYS')
len(df)

52

### Team Offensive Stats EDA

In [178]:


# Determine Pass / Rush Ratio
plays = len(df)
passing_df = df[df['action']=='rec']
rushing_df = df[df['action']=='rush']
# plays == len(passing_df)+len(rushing_df)
p_ratio = (len(passing_df)/plays)*100
r_ratio = (len(rushing_df)/plays)*100

# Avg Yds per Rec
p_avg = passing_df['yds'].mean()
# Avg Yds per Carry
r_avg = rushing_df['yds'].mean()

# Completion %
comp_df = passing_df[passing_df['completed']==True]
comp_pct = len(comp_df)/len(passing_df)

# Effective Carry %
eff_car_df = rushing_df[(rushing_df['yds']>5.0) | (rushing_df['converted']==True)]
eff_car_pct = len(eff_car_df)/len(rushing_df)


print('Plays : ' + str(plays))
print('Passing (' + str(round(p_ratio,2)) + '%) @ ' + str(round(comp_pct,2)) + 
      ' cmp for ' + str(round(p_avg,1)) + ' yd avg')
print('Rushing (' + str(round(r_ratio,2)) + '%) @ ' + str(round(eff_car_pct,2)) + 
      ' eff for ' + str(round(r_avg,1)) + ' yd avg')


Plays : 52
Passing (30.77%) @ 0.69 cmp for 9.1 yd avg
Rushing (69.23%) @ 0.58 eff for 5.3 yd avg


### Individual Analysis of Skill Players

In [179]:
skill_players_rec = passing_df['player'].unique().tolist()
skill_players_rush = rushing_df['player'].unique().tolist()


In [180]:
results = {}

for player in skill_players_rec:
    if player not in results.keys():
        results[player] = {}
    df_player = df[df['player']==player]   
    df_player_rec = df_player[df_player['action']=='rec']
    # Plays     
    results[player]['plays'] = len(df_player_rec)
    # Cmp 
    results[player]['targets'] = len(df_player_rec)
    df_player_cmp = df_player_rec[df_player_rec['completed']==True]
    results[player]['cmp'] = len(df_player_cmp)
    # Yds
    rec_yds = df_player_cmp['yds'].sum()
    results[player]['rec_yds'] = rec_yds
    
results
     

{'rutherford': {'plays': 7, 'targets': 7, 'cmp': 4, 'rec_yds': 66.0},
 'haynes': {'plays': 2, 'targets': 2, 'cmp': 2, 'rec_yds': 16.0},
 'singleton': {'plays': 4, 'targets': 4, 'cmp': 2, 'rec_yds': 19.0},
 'lane': {'plays': 2, 'targets': 2, 'cmp': 2, 'rec_yds': 31.0},
 'boyd': {'plays': 1, 'targets': 1, 'cmp': 1, 'rec_yds': 14.0}}

In [181]:
for player in skill_players_rush:
    if player not in results.keys():
        results[player] = {}
    df_player = df[df['player']==player]   
    df_player_rush = df_player[df_player['action']=='rush']
    # Plays    
    if 'plays' not in results[player].keys(): 
        results[player]['plays'] = len(df_player_rush)
    else:
        results[player]['plays'] += len(df_player_rush)
    # Effective Carries
    results[player]['car'] = len(df_player_rush)
    df_player_eff = df_player_rush[(df_player_rush['yds']>5.0) | (df_player_rush['converted']==True)]
    results[player]['eff_car'] = len(df_player_eff)
    # Yds
    rush_yds = df_player_rush['yds'].sum()
    results[player]['rush_yds'] = rush_yds

results


{'rutherford': {'plays': 8,
  'targets': 7,
  'cmp': 4,
  'rec_yds': 66.0,
  'car': 1,
  'eff_car': 1,
  'rush_yds': 9.0},
 'haynes': {'plays': 13,
  'targets': 2,
  'cmp': 2,
  'rec_yds': 16.0,
  'car': 11,
  'eff_car': 7,
  'rush_yds': 75.0},
 'singleton': {'plays': 5,
  'targets': 4,
  'cmp': 2,
  'rec_yds': 19.0,
  'car': 1,
  'eff_car': 1,
  'rush_yds': 10.0},
 'lane': {'plays': 2, 'targets': 2, 'cmp': 2, 'rec_yds': 31.0},
 'boyd': {'plays': 1, 'targets': 1, 'cmp': 1, 'rec_yds': 14.0},
 'king': {'plays': 15, 'car': 15, 'eff_car': 7, 'rush_yds': 54.0},
 'pyron': {'plays': 1, 'car': 1, 'eff_car': 1, 'rush_yds': 1.0},
 'alexander': {'plays': 7, 'car': 7, 'eff_car': 4, 'rush_yds': 41.0}}

### First Down Conversions

In [182]:
df_first = df[df['down']==1]
df_first_conv = df_first[df_first['converted']==True]
df_first_conv.sort_values(by='yds',ascending=False)

,opp,player,down,ytg,action,completed,yds,converted,field_pos
2,fsu,rutherford,1,10.0,rec,True,42.0,True,45
23,fsu,haynes,1,10.0,rush,NaN,36.0,True,25
0,fsu,haynes,1,10.0,rush,NaN,14.0,True,21
30,fsu,boyd,1,10.0,rec,True,14.0,True,11
1,fsu,singleton,1,10.0,rush,NaN,10.0,True,35
40,fsu,haynes,1,2.0,rush,NaN,2.0,True,98
5,fsu,pyron,1,1.0,rush,NaN,1.0,True,99
22,fsu,haynes,1,1.0,rush,NaN,1.0,True,99


### Third Down Conversions

In [183]:
df_third = df[df['down']==3]
df_third_conv = df_third[df_third['converted']==True]
df_third_conv.sort_values(by='yds',ascending=False)

,opp,player,down,ytg,action,completed,yds,converted,field_pos
43,fsu,haynes,3,7.0,rec,True,16.0,True,28
37,fsu,rutherford,3,12.0,rec,True,15.0,True,57
11,fsu,lane,3,5.0,rec,True,11.0,True,30
48,fsu,alexander,3,1.0,rush,NaN,2.0,True,67


### GT Territory

In [190]:
df_gt_terr = df[df['field_pos']<50]
df_gt_terr = df_gt_terr.sort_values(['down','action'],ascending=True)
df_gt_terr

,opp,player,down,ytg,action,completed,yds,converted,field_pos
2,fsu,rutherford,1,10.0,rec,True,42.0,True,45
9,fsu,singleton,1,10.0,rec,False,0.0,False,25
30,fsu,boyd,1,10.0,rec,True,14.0,True,11
33,fsu,rutherford,1,10.0,rec,True,5.0,False,45
41,fsu,rutherford,1,10.0,rec,True,4.0,False,25
0,fsu,haynes,1,10.0,rush,NaN,14.0,True,21
1,fsu,singleton,1,10.0,rush,NaN,10.0,True,35
6,fsu,haynes,1,10.0,rush,NaN,-6.0,False,31
12,fsu,haynes,1,10.0,rush,NaN,6.0,False,41
23,fsu,haynes,1,10.0,rush,NaN,36.0,True,25


### Opponent Territory

In [185]:
df_opp_terr = df[(df['field_pos']>=50)&(df['field_pos']<80)]
df_opp_terr.sort_values(by='yds',ascending=False)

,opp,player,down,ytg,action,completed,yds,converted,field_pos
39,fsu,king,2,5.0,rush,NaN,21.0,True,77
37,fsu,rutherford,3,12.0,rec,True,15.0,True,57
15,fsu,king,2,10.0,rush,NaN,12.0,True,63
51,fsu,singleton,3,17.0,rec,True,12.0,False,62
34,fsu,rutherford,2,5.0,rush,NaN,9.0,True,50
35,fsu,singleton,1,10.0,rec,True,7.0,False,59
16,fsu,alexander,1,10.0,rush,NaN,7.0,False,75
45,fsu,alexander,2,3.0,rush,NaN,7.0,True,51
47,fsu,haynes,2,7.0,rush,NaN,6.0,False,61
38,fsu,alexander,1,10.0,rush,NaN,5.0,False,72


### Red Zone 

In [186]:
df_red_zone = df[df['field_pos']>=80]
df_red_zone.sort_values(by='yds',ascending=False)

,opp,player,down,ytg,action,completed,yds,converted,field_pos
3,fsu,haynes,1,10.0,rush,NaN,9.0,False,87
19,fsu,king,2,6.0,rush,NaN,7.0,True,90
17,fsu,king,2,3.0,rush,NaN,4.0,True,82
18,fsu,haynes,1,10.0,rush,NaN,4.0,False,86
4,fsu,king,2,1.0,rush,NaN,3.0,True,96
40,fsu,haynes,1,2.0,rush,NaN,2.0,True,98
5,fsu,pyron,1,1.0,rush,NaN,1.0,True,99
20,fsu,alexander,1,3.0,rush,NaN,1.0,False,97
21,fsu,king,1,2.0,rush,NaN,1.0,False,98
22,fsu,haynes,1,1.0,rush,NaN,1.0,True,99
